In [107]:
#IMPORTS
import pandas as pd
from datetime import datetime

#CONFIGS
pd.options.display.max_columns = 100

# Data accessment and cleaning
A data dictionary of the listings.csv dataset can be found [here](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=1322284596).

In [223]:
# import listings files
listings_df_202303 = pd.read_csv("input-data\\202303-listings.csv")
listings_df_202212 = pd.read_csv("input-data\\202212-listings.csv")
listings_df_202209 = pd.read_csv("input-data\\202209-listings.csv")
listings_df_202206 = pd.read_csv("input-data\\202206-listings.csv")

listings_df_202303["file_name"] = "202303-listings.csv"
listings_df_202212["file_name"] = "202212-listings.csv"
listings_df_202209["file_name"] = "202209-listings.csv"
listings_df_202206["file_name"] = "202206-listings.csv"

listings_df_ls = [listings_df_202303, listings_df_202212, listings_df_202209, listings_df_202206]

## First cleaning
### Columns removal
In this step I will remove columns which do not include relevant information for this project. I considered the following types of columns as not relevant:
1. It contains meta information about the listing like URL or description (this one could be important if NLP analysis was in scope);
2. Technical columns like scrape ID or date;
3. Personal data like name of the host;
4. Host-related data, since it is not relevant for the purpose of this analysis;
5. Columns with a high number of nulls;
6. Other columns which I considered too specific for the purpose of this analysis.

In [202]:
listings_df_ls[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12410 entries, 0 to 12409
Data columns (total 76 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            12410 non-null  int64  
 1   listing_url                                   12410 non-null  object 
 2   scrape_id                                     12410 non-null  int64  
 3   last_scraped                                  12410 non-null  object 
 4   source                                        12410 non-null  object 
 5   name                                          12410 non-null  object 
 6   description                                   12405 non-null  object 
 7   neighborhood_overview                         7608 non-null   object 
 8   picture_url                                   12410 non-null  object 
 9   host_id                                       12410 non-null 

In [224]:
def drop_cols(df, col_list):
    """
    Description
    -----------
    This function drops a specified list of columns for a given pandas DataFrame. The columns in the list must do not necessarily need to exist in the DataFrame.  
    
    Arguments
    ---------
    df: pandas DataFrame
        a pandas DataFrame from which the columns should be removed.

    col_list: list
        a list of columns to be removed from the pandas DataFrame.

    Returns
    -------
    df: pandas DataFrame
        a pandas DataFrame with the columns removed which were found in the list.
    """   
    
    for col in col_list:
        if col in df.columns:
            df = df.drop(col, axis=1)
            
    return df

In [225]:
drop_col_ls = ['listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
               'description', 'neighborhood_overview', 'picture_url', 'host_url',
               'host_name','host_since', 'host_location', 'host_about','host_response_time', 
               'host_response_rate', 'host_acceptance_rate','host_thumbnail_url', 'host_picture_url',
               'host_neighbourhood', 'host_total_listings_count','host_verifications','host_has_profile_pic', 
               'host_listings_count', 'host_identity_verified', 'neighbourhood','bathrooms','minimum_nights',
               'maximum_nights','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights',
               'maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','calendar_updated',
               'has_availability','availability_30','availability_60','availability_365','number_of_reviews_ltm',
               'number_of_reviews_l30d','first_review','last_review','review_scores_accuracy',
               'review_scores_cleanliness','review_scores_checkin','review_scores_communication',
               'review_scores_location','review_scores_rating','instant_bookable','license',
               'calculated_host_listings_count','calculated_host_listings_count_entire_homes', 
               'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms',
               'reviews_per_month']

# remove the columns for each DataFrame in the list
for i in range(0, len(listings_df_ls)):
    listings_df_ls[i] = drop_cols(listings_df_ls[i], drop_col_ls)

## Dealing with missing data

In [229]:
# Print the number of nulls per column for each DataFrame
for i in range(0, len(listings_df_ls)):
    print(f"""TABLE NAME: {listings_df_ls[i]["file_name"].unique()[0]}""")
    print("------------------------------------------------------------\n")
    print(listings_df_ls[i].isnull().sum())
    print("\n------------------------------------------------------------\n")

TABLE NAME: 202303-listings.csv
------------------------------------------------------------

id                                 0
host_id                            0
host_is_superhost                  0
neighbourhood_cleansed             0
neighbourhood_group_cleansed       0
latitude                           0
longitude                          0
property_type                      0
room_type                          0
accommodates                       0
bathrooms_text                    14
bedrooms                           0
beds                               0
amenities                          0
price                              0
availability_90                    0
calendar_last_scraped              0
number_of_reviews                  0
review_scores_value             1622
file_name                          0
dtype: int64

------------------------------------------------------------

TABLE NAME: 202212-listings.csv
----------------------------------------------------------

It is possible to see that list of columns with missing data is similar for each DataFrame, so the chosen approaches will be used for all of the DataFrames.

For the purpose of the exploratory data analysis I decided to __address__ the following columns:
- bedrooms
- beds

For the purpose of the exploratory data analysis I decided to __ignore__ the following columns:
- host_is_superhost
- bathrooms_text
- review_scores_value

#### Procedure
Since the column _accomodates_ is always filled I simply used the __mode__ of the available data within the column to be treated while accounting for each corresponding value inside the column _accomodates_.

In [227]:
def fill_data_with_mode(df, ref_col, col_list):
    """
    Description
    -----------
    This function fills in missing data for a given list of columns in a pandas DataFrame based on the mode available in the missing column for a particular group of values available on a specified reference column.
    
    Arguments
    ---------
    df: pandas DataFrame
        a pandas DataFrame in which the missing data will be filled.
        
    ref_col: string
        the name of the reference column, from which the value group will be used to calculate the mode.
        
    col_list: list
        a list of columns with missing data.

    Returns
    -------
    df: pandas DataFrame
        a pandas DataFrame with the filled missing data.
    """   
    
    for col in col_list:
        mode_per_group = df.groupby(ref_col)[col].transform(lambda x: x.mode()[0])
        df[col] = df[col].fillna(mode_per_group)
    
    return df

In [228]:
ref_col = "accommodates"
missing_col_ls = ["bedrooms","beds"]

# fill in the missing data
for i in range(0, len(listings_df_ls)):
    listings_df_ls[i] = fill_data_with_mode(listings_df_ls[i],ref_col,missing_col_ls)

In [232]:
# quality check for missing data
print(f"""Missing data in bedrooms: {listings_df_ls[0]["bedrooms"].isnull().sum()}""")
print(f"""Missing data in bedrooms: {listings_df_ls[0]["beds"].isnull().sum()}""")

Missing data in bedrooms: 0
Missing data in bedrooms: 0


# Data quality Assessment

In [235]:
listings_df_ls[0].tail()

,id,host_id,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,availability_90,calendar_last_scraped,number_of_reviews,review_scores_value,file_name
12405,649818115035943903,464382071,t,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",PORTO,41.145993,-8.613339,Entire loft,Entire home/apt,2,1 bath,1.0,1.0,"[""City skyline view"", ""Wine glasses"", ""Bidet"",...",$148.00,31,2023-03-18,15,4.80,202303-listings.csv
12406,24334174,183600824,t,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",PORTO,41.147650,-8.616640,Entire rental unit,Entire home/apt,8,2 baths,3.0,4.0,"[""Patio or balcony"", ""Wine glasses"", ""Room-dar...",$102.00,24,2023-03-18,137,4.77,202303-listings.csv
12407,14182656,85929808,t,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",PORTO,41.144130,-8.613510,Entire rental unit,Entire home/apt,6,1.5 baths,2.0,3.0,"[""Patio or balcony"", ""City skyline view"", ""Cof...",$150.00,0,2023-03-18,91,4.67,202303-listings.csv
12408,2643760,4287899,t,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",PORTO,41.144070,-8.612460,Entire rental unit,Entire home/apt,2,1 bath,1.0,1.0,"[""Patio or balcony"", ""Wine glasses"", ""Bidet"", ...",$70.00,34,2023-03-18,231,4.76,202303-listings.csv
12409,24408708,184264583,f,Aguçadoura e Navais,PÓVOA DE VARZIM,41.425450,-8.776940,Entire home,Entire home/apt,10,2 baths,3.0,5.0,"[""Crib"", ""Wine glasses"", ""Bidet"", ""Room-darken...",$95.00,53,2023-03-18,27,4.44,202303-listings.csv


__Notes__:

__Observation__: Since I am going to focus the analysis of ocupancy rates baed on the column _availability_90_, I wanted to make a quick quality check to see if the data matches with the day-by-day availabiliy data in the calendar.csv for a speficic listing ID. I simply chose the first ID available: 840535633089695952.

In [105]:
#print 90-days' availability data in the 202303-listings.csv
listings_df_202303[listings_df_202303["id"] == 840535633089695952][["id","availability_90"]].head()

,id,availability_90
0,840535633089695952,72


In [104]:
#import calendar data for 2023
calender_df_202303 = pd.read_csv("input-data\\202303-calendar.csv")
calender_df_202303['date'] = pd.to_datetime(calender_df_2023['date'])

#filter the date for the listing id and the next 90 days
start_date = min(calender_df_202303['date']) #date of the extract
end_date = start_date + pd.DateOffset(days=90) #90 days after

id_filter = calender_df_202303["listing_id"] == 840535633089695952
date_filter = (calender_df_202303["date"] >= start_date) & (calender_df_202303["date"] < end_date)
combined_filter = id_filter & date_filter

filtered_calender_df_202303 = calender_df_202303[combined_filter]

nr_available_days = len(filtered_calender_df_202303[filtered_calender_df_202303["available"] == "t"])
print(f"""The listing id {str(filtered_calender_df_202303["listing_id"].unique()[0])} was {nr_available_days} days available between {str(start_date)[:10]} and {str(end_date)[:10]}.""")

The listing id 840535633089695952 was 72 days available between 2023-03-17 and 2023-06-15.
